In [ ]:
from textgcn import TextGCN
import pandas as pd
import torch
device='cpu'

In [ ]:
#Read the data
dev=pd.read_csv('../data/dev.csv')
train=pd.read_csv('../data/train.csv')
test=pd.read_csv('../data/test.csv')

In [ ]:
#Remove punctuations,numbers and emojis from text
import re 

#Removing punctuations
train['text']=train['text'].str.replace('[^\w\s]','')
dev['text']=dev['text'].str.replace('[^\w\s]','')
test['text']=test['text'].str.replace('[^\w\s]','')

#Removing numbers
train['text']=train['text'].str.replace('\d+','')
dev['text']=dev['text'].str.replace('\d+','')
test['text']=test['text'].str.replace('\d+','')

#Removing usernames(@user)
train['text']=train['text'].str.replace('@\w+','')

#Removing emojis
train['text']=train['text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
dev['text']=dev['text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
test['text']=test['text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

#Removing urls
train['text']=train['text'].str.replace(r'^https?:\/\/.*[\r\n]*', '', flags=re.MULTILINE)
dev['text']=dev['text'].str.replace(r'^https?:\/\/.*[\r\n]*', '', flags=re.MULTILINE)
test['text']=test['text'].str.replace(r'^https?:\/\/.*[\r\n]*', '', flags=re.MULTILINE)


In [ ]:
stopwords=[]
#Read TamilStopWords.txt file and append each line to stopwords list
with open('TamilStopWords.txt','r',encoding='utf-8') as f:
    for line in f:
        stopwords.append(line.strip())



In [ ]:
def vocabi(train,test,dev):
    #Remove stop words of tamil language
    vocab={}
    for i in train['text']:
        for j in i.split():
            if j not in vocab:
                vocab[j]=1
            else:
                vocab[j]+=1
    for i in test['text']:
        for j in i.split():
            if j not in vocab:
                vocab[j]=1
            else:
                vocab[j]+=1
    for i in dev['text']:
        for j in i.split():
            if j not in vocab:
                vocab[j]=1
            else:
                vocab[j]+=1
    vocabul={}
    for i in vocab:
        if i not in stopwords:
            vocabul[i]=vocab[i]
    vocab=vocabul

    for i in train['text']:
        sentence=''
        for j in i.split():
            if j in vocab:
                sentence+=' '+j
        train['text'].replace(i,sentence,inplace=True)
    for i in test['text']:
        sentence=''
        for j in i.split():
            if j not in vocab:
                pass
            else:
                sentence+=' '+j
        test['text'].replace(i,sentence,inplace=True)
    for i in dev['text']:
        sentence=''
        for j in i.split():
            if j not in vocab:
                pass
            else:
                sentence+=' '+j
        dev['text'].replace(i,sentence,inplace=True)
    vocabulary={}
    i=0
    for j in vocab:
        vocabulary[j]=i
        i+=1
    return vocabulary
    



In [ ]:
vocab=vocabi(train,test,dev)

In [ ]:
def find_pmi(data1,data2,data3,vocab):
    import math
    sliding_window=5
    pmi={}
    number_of_windows=0
    count_of_words={}
    count_of_word_pairs={}
    #Loop over all the documents in data1 and data2
    for i in data1:
        words=i.split()
        for j in range(len(words)):
            if vocab[words[j]] not in count_of_words:
                count_of_words[vocab[words[j]]]=1
            else:
                count_of_words[vocab[words[j]]]+=1
            for k in range(j+1,min(j+sliding_window,len(words))):
                a=min(vocab[words[j]],vocab[words[k]])
                b=max(vocab[words[j]],vocab[words[k]])
                if (a,b) not in count_of_word_pairs:
                    count_of_word_pairs[(a,b)]=1
                else:
                    count_of_word_pairs[(a,b)]+=1
        number_of_windows+=1
    for i in data2:
        words=i.split()
        for j in range(len(words)):
            if vocab[words[j]] not in count_of_words:
                count_of_words[vocab[words[j]]]=1
            else:
                count_of_words[vocab[words[j]]]+=1
            for k in range(j+1,min(j+sliding_window,len(words))):
                a=min(vocab[words[j]],vocab[words[k]])
                b=max(vocab[words[j]],vocab[words[k]])
                if (a,b) not in count_of_word_pairs:
                    count_of_word_pairs[(a,b)]=1
                else:
                    count_of_word_pairs[(a,b)]+=1
        number_of_windows+=1
    for i in data3:
        words=i.split()
        for j in range(len(words)):
            if vocab[words[j]] not in count_of_words:
                count_of_words[vocab[words[j]]]=1
            else:
                count_of_words[vocab[words[j]]]+=1
            for k in range(j+1,min(j+sliding_window,len(words))):
                a=min(vocab[words[j]],vocab[words[k]])
                b=max(vocab[words[j]],vocab[words[k]])
                if (a,b) not in count_of_word_pairs:
                    count_of_word_pairs[(a,b)]=1
                else:
                    count_of_word_pairs[(a,b)]+=1
        number_of_windows+=1
    
    for i in count_of_word_pairs:
        pmi[i]=math.log((count_of_word_pairs[(i[0],i[1])]*number_of_windows)/(count_of_words[i[0]]*count_of_words[i[1]]))
        pmi[(i[1],i[0])]=pmi[i]
    return pmi

def find_tfidf(data1,data2,data3,vocab):
    import math
    tfidf={}
    number_of_documents=0
    count_of_words={}
    max_count_of_words={}
    tf={}
    idf={}
    count=0
    for i in data1:
        max_count_of_words[len(vocab)+count]=0
        words=i.split()
        for j in range(len(words)):
            if vocab[words[j]] not in count_of_words:
                count_of_words[vocab[words[j]]]=1
                max_count_of_words[len(vocab)+count]=max(max_count_of_words[len(vocab)+count],1)
            else:
                count_of_words[vocab[words[j]]]+=1
                max_count_of_words[len(vocab)+count]=max(max_count_of_words[len(vocab)+count],count_of_words[vocab[words[j]]])
        number_of_documents+=1
        count+=1
    for i in data2:
        max_count_of_words[len(vocab)+count]=0
        words=i.split()
        for j in range(len(words)):
            if vocab[words[j]] not in count_of_words:
                count_of_words[vocab[words[j]]]=1
                max_count_of_words[len(vocab)+count]=max(max_count_of_words[len(vocab)+count],1)
            else:
                count_of_words[vocab[words[j]]]+=1
                max_count_of_words[len(vocab)+count]=max(max_count_of_words[len(vocab)+count],count_of_words[vocab[words[j]]])
        number_of_documents+=1
        count+=1
    for i in data3:
        max_count_of_words[len(vocab)+count]=0
        words=i.split()
        for j in range(len(words)):
            if vocab[words[j]] not in count_of_words:
                count_of_words[vocab[words[j]]]=1
                max_count_of_words[len(vocab)+count]=max(max_count_of_words[len(vocab)+count],1)
            else:
                count_of_words[vocab[words[j]]]+=1
                max_count_of_words[len(vocab)+count]=max(max_count_of_words[len(vocab)+count],count_of_words[vocab[words[j]]])
        number_of_documents+=1
        count+=1
    
    for i in vocab:
        count=0
        for j in range(len(data1)):
            word=data1[j].split()
            for k in word:
                if k==i:
                    if (vocab[k],len(vocab)+j) not in tf:
                        tf[(vocab[k],len(vocab)+j)]=1
                        count+=1
                    else:
                        tf[(vocab[k],len(vocab)+j)]+=1
                        count+=1
        for j in range(len(data2)):
            word=data2[j].split()
            for k in word:
                if k==i:
                    if (vocab[k],len(vocab)+j+len(data1)) not in tf:
                        tf[(vocab[k],len(vocab)+j+len(data1))]=1
                        count+=1
                    else:
                        tf[(vocab[k],len(vocab)+j+len(data1))]+=1
                        count+=1
        for j in range(len(data3)):
            word=data3[j].split()
            for k in word:
                if k==i:
                    if (vocab[k],len(vocab)+j+len(data1)+len(data2)) not in tf:
                        tf[(vocab[k],len(vocab)+j+len(data1)+len(data2))]=1
                        count+=1
                    else:
                        tf[(vocab[k],len(vocab)+j+len(data1)+len(data2))]+=1
                        count+=1
        if count!=0:
            idf[vocab[i]]=math.log(number_of_documents/count)

    for (a,b) in tf:
        tfidf[(a,b)]=(tf[(a,b)]*idf[a])/max_count_of_words[b]
        tfidf[(b,a)]=tfidf[(a,b)]
    return tfidf

In [ ]:
def create_matrix(data1,data2,data3,vocab):
    #Find number of unique words in data
    matrix=torch.zeros(len(vocab)+len(data1)+len(data2)+len(data3),len(vocab)+len(data1)+len(data2)+len(data3)).to(device)
    pmi=find_pmi(data1,data2,data3,vocab)
    tfidf=find_tfidf(data1,data2,data3,vocab)
    for i in range(len(vocab)+len(data1)+len(data2)+len(data3)):
        matrix[i][i]=1
    for i in pmi:
            matrix[i[0]][i[1]]=pmi[i]
    for i in tfidf:
        matrix[i[1]][i[0]]=tfidf[i]

    #Write the length of data1 and data2 in a file
    f=open('length.txt','w')
    f.write(str(len(data1)))
    f.write('\n')
    f.write(str(len(data2)))
    f.write('\n')
    f.write(str(len(data3)))
    f.close()
    return matrix.to(device)


In [ ]:
adj=create_matrix(train['text'],dev['text'],test['text'],vocab)

In [ ]:
#Find D
D=torch.zeros(len(vocab)+len(train['text'])+len(dev['text'])+len(test['text']),len(vocab)+len(train['text'])+len(dev['text'])+len(test['text'])).to(device)
for i in range(len(vocab)+len(train['text'])+len(dev['text'])+len(test['text'])):
    D[i][i]=torch.sum(adj[i])
D=D.to(device)
#Make matrix inverse of D
D=torch.inverse(D)
D=torch.pow(D,0.5)
adj=torch.mm(torch.mm(D,adj),D)

In [ ]:
torch.save(adj,'../adj.pt')